<a href="https://colab.research.google.com/github/muhozag/sparklearning/blob/trees/tree2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tree Algorithms Example: Public vs. Private University Prediction**

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree').getOrCreate()

In [2]:
data = spark.read.csv('College.csv',inferSchema=True,header=True)

In [3]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [4]:
data.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [5]:
from pyspark.ml.feature import VectorAssembler

In [6]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [7]:
assembler = VectorAssembler(
    inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'], outputCol='features'
)

In [8]:
output = assembler.transform(data)

In [10]:
from pyspark.ml.feature import StringIndexer

In [11]:
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')

In [12]:
output_fixed = indexer.fit(output).transform(output)

In [13]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [14]:

final_data = output_fixed.select('features', 'PrivateIndex')

In [15]:
train_data,test_data = final_data.randomSplit([0.75,0.25])

In [16]:
from pyspark.ml.classification import (DecisionTreeClassifier,
                                       GBTClassifier, RandomForestClassifier)

In [52]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(numTrees=200, labelCol='PrivateIndex', featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [53]:
from pyspark.ml import Pipeline

In [54]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [55]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [56]:
dtc_preds.show()

+--------------------+------------+-------------+--------------------+----------+
|            features|PrivateIndex|rawPrediction|         probability|prediction|
+--------------------+------------+-------------+--------------------+----------+
|[191.0,165.0,63.0...|         0.0|  [313.0,0.0]|           [1.0,0.0]|       0.0|
|[232.0,182.0,99.0...|         0.0|    [4.0,6.0]|           [0.4,0.6]|       1.0|
|[235.0,217.0,121....|         0.0|   [18.0,0.0]|           [1.0,0.0]|       0.0|
|[244.0,198.0,82.0...|         0.0|  [313.0,0.0]|           [1.0,0.0]|       0.0|
|[280.0,143.0,79.0...|         0.0|    [6.0,0.0]|           [1.0,0.0]|       0.0|
|[285.0,280.0,208....|         1.0|   [28.0,1.0]|[0.96551724137931...|       0.0|
|[346.0,274.0,146....|         0.0|  [313.0,0.0]|           [1.0,0.0]|       0.0|
|[368.0,317.0,159....|         0.0|  [313.0,0.0]|           [1.0,0.0]|       0.0|
|[427.0,385.0,143....|         0.0|  [313.0,0.0]|           [1.0,0.0]|       0.0|
|[434.0,321.0,14

In [57]:
rfc_preds.show()

+--------------------+------------+--------------------+--------------------+----------+
|            features|PrivateIndex|       rawPrediction|         probability|prediction|
+--------------------+------------+--------------------+--------------------+----------+
|[191.0,165.0,63.0...|         0.0|[179.457123079618...|[0.89728561539809...|       0.0|
|[232.0,182.0,99.0...|         0.0|[149.006130346732...|[0.74503065173366...|       0.0|
|[235.0,217.0,121....|         0.0|[196.590170493853...|[0.98295085246926...|       0.0|
|[244.0,198.0,82.0...|         0.0|[193.679373661863...|[0.96839686830931...|       0.0|
|[280.0,143.0,79.0...|         0.0|[135.772226179138...|[0.67886113089569...|       0.0|
|[285.0,280.0,208....|         1.0|[104.389638473699...|[0.52194819236849...|       0.0|
|[346.0,274.0,146....|         0.0|[191.412278380512...|[0.95706139190256...|       0.0|
|[368.0,317.0,159....|         0.0|[195.321094399774...|[0.97660547199887...|       0.0|
|[427.0,385.0,143....

In [58]:
gbt_preds.show()

+--------------------+------------+--------------------+--------------------+----------+
|            features|PrivateIndex|       rawPrediction|         probability|prediction|
+--------------------+------------+--------------------+--------------------+----------+
|[191.0,165.0,63.0...|         0.0|[1.54296120893688...|[0.95630830919150...|       0.0|
|[232.0,182.0,99.0...|         0.0|[-0.5753938065146...|[0.24034525172021...|       1.0|
|[235.0,217.0,121....|         0.0|[1.53936824231755...|[0.95600707485356...|       0.0|
|[244.0,198.0,82.0...|         0.0|[1.54030021764277...|[0.95608540144862...|       0.0|
|[280.0,143.0,79.0...|         0.0|[0.91620658435534...|[0.86204895299728...|       0.0|
|[285.0,280.0,208....|         1.0|[0.79313394535674...|[0.83009037335726...|       0.0|
|[346.0,274.0,146....|         0.0|[1.54280453455569...|[0.95629521475354...|       0.0|
|[368.0,317.0,159....|         0.0|[1.54369909996036...|[0.95636992988255...|       0.0|
|[427.0,385.0,143....

In [59]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [60]:
my_binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [61]:
my_binary_eval.evaluate(dtc_preds)

0.9126465884861408

In [62]:
my_binary_eval.evaluate(rfc_preds)

0.9736140724946692

In [63]:
my_binary_eval.evaluate(gbt_preds)

0.9268390191897652

In [64]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [65]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex',
                                             metricName='accuracy')

In [66]:
acc_eval.evaluate(dtc_preds)

0.8947368421052632

In [68]:
acc_eval.evaluate(rfc_preds)

0.9473684210526315

In [70]:
acc_eval.evaluate(gbt_preds)

0.8947368421052632